In [ ]:
import tensorflow as tf
import wandb
from wandb.integration.keras import WandbMetricsLogger
from datetime import datetime, timedelta
from airflow import DAG
from airflow.operators.python import PythonOperator
import numpy as np


In [ ]:
# Wandb 설정

wandb.login()
WANDB_PROJECT = aa
WANDB_ENTITY = bb

In [ ]:
def load_and_preprocess_data():
    # TensorFlow.keras로 MNISTS 데이터셋 로드하고 전처리해주세요!
    #[[YOUR CODE]]
    return (x_train, y_train), (x_test, y_test)


In [ ]:
def create_model(learning_rate=0.001, conv1_filters=32, conv2_filters=64):
    """CNN 모델을 간단하게 생성해주세요"""
    #[[YOUR CODE]]
    
    return model

In [ ]:
def train_model(**context):
    """모델 학습 및 W&B 로깅"""
    # W&B 초기화
    wandb.init(
        #[[YOUR CODE]]
    )
    
    # 데이터 로드
    (x_train, y_train), (x_test, y_test) = load_and_preprocess_data()
    
    # 모델 생성
    model = create_model(
        #[[YOUR CODE]]
    )
    
    # 모델 학습
    model.fit(

        #[[YOUR CODE]]
    )
    
    # 모델 평가
    test_loss, test_accuracy = model.evaluate(x_test, y_test)
    print(f"Test accuracy: {test_accuracy}")
    
    # W&B에 최종 메트릭 기록 (test_loss, test_accuracy)
    wandb.log({
        #[[YOUR CODE]]
    })
    
    # 모델 저장
    model.save(f"mnist_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}.keras")
    
    wandb.finish()

In [ ]:
def hyperparameter_sweep():
    """W&B를 사용한 하이퍼파라미터 튜닝"""
    sweep_config = {
        'method': 'random',
        'metric': {'name': 'val_accuracy', 'goal': 'maximize'},
        'parameters': {
            'learning_rate': {'values': [0.001, 0.01, 0.0001]},
            'conv1_filters': {'values': [16, 32, 64]},
            'conv2_filters': {'values': [32, 64, 128]},
            'batch_size': {'values': [64, 128, 256]}
        }
    }
    
    sweep_id = wandb.sweep(sweep_config, project=WANDB_PROJECT)
    wandb.agent(sweep_id, train_model, count=5)

In [ ]:
# Airflow DAG 정의

import pendulum  # Airflow는 pendulum을 사용합니다

local_tz = pendulum.timezone("Asia/Seoul")  # 한국 시간대 사용

default_args = {
    'owner': 'airflow',
    'start_date': datetime(2024, 1, 1, tzinfo=local_tz),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}



dag = DAG(
    'mnist_training_pipeline',
    default_args=default_args,
    description='MNIST 학습 파이프라인',
    schedule = '@daily',
    catchup=False
)

# DAG 태스크 정의
preprocessing_task = PythonOperator(
    task_id='load_and_preprocess_data',
    python_callable=load_and_preprocess_data,
    dag=dag
)

training_task = PythonOperator(
    task_id='train_model',
    python_callable=train_model,
    dag=dag
)

hyperparameter_tuning_task = PythonOperator(
    task_id='hyperparameter_tuning',
    python_callable=hyperparameter_sweep,
    dag=dag
)

# 태스크 의존성 설정
preprocessing_task >> training_task >> hyperparameter_tuning_task

# Jupyter Notebook에서 직접 실행하기 위한 코드
if __name__ == "__main__":
    # 단일 실험 실행
    train_model()

    # hyperparameter_sweep()

In [ ]:
execution_date = datetime(2024, 11, 10, tzinfo=local_tz)
dag.test(execution_date=execution_date)